In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [13]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)

In [14]:
device=0
log_steps=1
num_layers=2
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [15]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [16]:
adj_t = data.adj_t.to(device)

In [17]:
torch.manual_seed(12345)

In [18]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [19]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [20]:
predictor = LinkPredictor(hidden_channels, hidden_channels, 1,
                          num_layers, dropout).to(device)

In [21]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [22]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [23]:
model.reset_parameters()
predictor.reset_parameters()

In [24]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [25]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [26]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 0.9371
Epoch: 04, Loss: 0.6370
Epoch: 06, Loss: 0.4759
Epoch: 08, Loss: 0.3767
Epoch: 10, Loss: 0.3145
Epoch: 12, Loss: 0.2664
Epoch: 14, Loss: 0.2313
Epoch: 16, Loss: 0.2066
Epoch: 18, Loss: 0.1867
Epoch: 20, Loss: 0.1717
Epoch: 22, Loss: 0.1592
Epoch: 24, Loss: 0.1471
Epoch: 26, Loss: 0.1386
Epoch: 28, Loss: 0.1295
Epoch: 30, Loss: 0.1230
Epoch: 32, Loss: 0.1153
Epoch: 34, Loss: 0.1096
Epoch: 36, Loss: 0.1041
Epoch: 38, Loss: 0.0995
Epoch: 40, Loss: 0.0953
Epoch: 42, Loss: 0.0923
Epoch: 44, Loss: 0.0879
Epoch: 46, Loss: 0.0858
Epoch: 48, Loss: 0.0827
Epoch: 50, Loss: 0.0818
Epoch: 52, Loss: 0.0790
Epoch: 54, Loss: 0.0762
Epoch: 56, Loss: 0.0743
Epoch: 58, Loss: 0.0723
Epoch: 60, Loss: 0.0714
Epoch: 62, Loss: 0.0698
Epoch: 64, Loss: 0.0684
Epoch: 66, Loss: 0.0671
Epoch: 68, Loss: 0.0656
Epoch: 70, Loss: 0.0644
Epoch: 72, Loss: 0.0632
Epoch: 74, Loss: 0.0623
Epoch: 76, Loss: 0.0619
Epoch: 78, Loss: 0.0603
Epoch: 80, Loss: 0.0599
Epoch: 82, Loss: 0.0592
Epoch: 84, Loss:

In [27]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
    (1): SAGEConv(50, 50)
  )
)

In [28]:
emb.weight

Parameter containing:
tensor([[ 0.2425, -0.0550,  0.3195,  ..., -0.3155, -0.2970, -0.0415],
        [-0.0584,  0.2655, -0.2138,  ...,  0.0009,  0.0939,  0.2854],
        [ 0.2017,  0.0092,  0.1748,  ..., -0.0752,  0.0584, -0.1292],
        ...,
        [ 0.0982, -0.0092, -0.0751,  ...,  0.4055, -0.0274,  0.0576],
        [ 0.0441,  0.0335,  0.0022,  ...,  0.1451,  0.1604,  0.1819],
        [ 0.2972,  0.2201, -0.0575,  ...,  0.1719,  0.2206,  0.3092]],
       device='cuda:0', requires_grad=True)

In [29]:
h = model(emb.weight, adj_t)

In [30]:
h

tensor([[ 0.0697, -0.3768, -0.2769,  ..., -0.3470, -0.7486, -0.3634],
        [-0.1933, -2.7483, -1.1975,  ...,  1.2924,  2.9306,  0.2571],
        [-0.2040, -0.3921, -0.6902,  ...,  1.1801, -0.2978,  0.0182],
        ...,
        [ 0.1239, -0.3540, -0.5780,  ...,  0.6596, -1.2757,  0.4012],
        [-0.0319,  0.0711, -0.6963,  ...,  0.5472, -1.0973,  0.2865],
        [-0.3994,  0.4128, -1.2209,  ...,  2.2971,  0.9820,  1.2141]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [31]:
predictor(h[0], h[411])

tensor([0.9994], device='cuda:0', grad_fn=<SigmoidBackward>)

In [32]:
predictor(h[0], h[2])

tensor([4.5428e-06], device='cuda:0', grad_fn=<SigmoidBackward>)

In [33]:
(h[0] * h[2]).sum()

tensor(14.6862, device='cuda:0', grad_fn=<SumBackward0>)

In [34]:
(h[0] * h[411]).sum()

tensor(33.5396, device='cuda:0', grad_fn=<SumBackward0>)

In [35]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_15_h.npy', h.cpu().detach().numpy())

In [36]:
torch.save(model, '../../models/graphsage_link_pred/15_model.pt')

In [37]:
torch.save(emb, '../../models/graphsage_link_pred/15_embedding.pt')

In [38]:
torch.save(predictor, '../../models/graphsage_link_pred/15_predictor.pt')